In [2]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
from air_hockey_challenge.framework.challenge_core import ChallengeCore
from air_hockey_challenge.framework.agent_base import AgentBase
from examples.control.hitting_agent import build_agent, HittingAgent

from mushroom_rl.utils.dataset import parse_dataset, select_random_samples
from mushroom_rl.policy import GaussianTorchPolicy

import torch
import torch.nn as nn

from tqdm import tqdm

import pickle

import numpy as np

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(f"Cuda: {use_cuda}")


Cuda: False


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

class MushroomRLTrajectoryDataset(Dataset):
    def __init__(self, mdp, agent, n_episodes):
        """
        Initialize the MushroomRLTrajectoryDataset.

        Args:
            mdp (mushroom_rl.environments.Environment): the MDP (Markov Decision Process).
            agent: the agent to evaluate.
        """
        self.core = ChallengeCore(agent, mdp)
                # Initialize empty lists for each data type
        state_list, action_list, reward_list, next_state_list, absorbing_list, last_list = [], [], [], [], [], []

        for i in tqdm(range(n_episodes)):
            # Evaluate one episode at a time
            trajectory = self.core.evaluate(n_episodes=1, render=False)
            state, action, reward, next_state, absorbing, last = parse_dataset(trajectory)

            # Append the data from the current episode
            state_list.append(torch.from_numpy(state).to(device))
            action_list.append(torch.from_numpy(action).to(device))
            reward_list.append(torch.from_numpy(reward).to(device))
            next_state_list.append(torch.from_numpy(next_state).to(device))
            absorbing_list.append(torch.from_numpy(absorbing).to(device))
            last_list.append(torch.from_numpy(last).to(device))

        # Concatenate the data from all episodes
        self.state = torch.cat(state_list, dim=0)
        self.action = torch.cat(action_list, dim=0)
        self.reward = torch.cat(reward_list, dim=0)
        self.next_state = torch.cat(next_state_list, dim=0)
        self.absorbing = torch.cat(absorbing_list, dim=0)
        self.last = torch.cat(last_list, dim=0)

        self.length = self.state.shape[0]

    def __len__(self):
        """
        Return the total number of state-action-reward-next_state tuples in the dataset.

        Returns:
            int: the length of the dataset.
        """
        return self.length

    def __getitem__(self, index):
        """
        Get the state-action-reward-next_state tuple at the specified index.

        Args:
            index (int): the index of the desired tuple.

        Returns:
            dict: a dictionary containing state, action, reward, and next_state.
        """

        return {
            'state': self.state[index],
            'action': self.action[index],
            'reward': self.reward[index],
            'next_state': self.next_state[index],
            'absorbing': self.absorbing[index],
            'last': self.last[index],
        }



In [4]:
class Network(nn.Module):
    def __init__(self, input_shape, output_shape, n_features, **kwargs):
        super(Network, self).__init__()

        self._h1 = nn.Linear(input_shape[0], n_features)
        self._h2 = nn.Linear(n_features, n_features)
        self._h3 = nn.Linear(n_features, output_shape[0])

        nn.init.xavier_uniform_(self._h1.weight,
                                gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self._h2.weight,
                                gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self._h3.weight,
                                gain=nn.init.calculate_gain('linear'))

    def forward(self, obs, **kwargs):
        features1 = torch.tanh(self._h1(torch.squeeze(obs, 1).float()))
        features2 = torch.tanh(self._h2(features1))
        return self._h3(features2)


class BCAgent(AgentBase):
    def __init__(self, env_info, policy, **kwargs):
        super().__init__(env_info, **kwargs)
        self.policy = policy

    def reset(self):
        pass

    def draw_action(self, observation):
        return self.policy.draw_action(observation).reshape(2,3)


In [55]:
def train_dagger_agent(learner_policy, expert_policy, mdp, dataset, num_iterations=10, batch_size=64, lr=0.001):
    # Create data loader
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Set up the optimizer and loss function
    optimizer = optim.Adam(learner_policy.parameters(), lr=lr)
    loss_function = nn.MSELoss()

    # Train the initial policy on collected data
    for epoch in range(num_iterations):
        for i, data in enumerate(data_loader):
            optimizer.zero_grad()

            states = data['state'].float()

            # Get expert actions for each state in the batch
            expert_actions = []
            for state in states:
                expert_policy.reset()
                state_np = state.cpu().numpy()
                expert_action = expert_policy.draw_action(state_np).astype(np.float32)
                expert_actions.append(torch.from_numpy(expert_action))
            expert_actions = torch.stack(expert_actions).reshape((batch_size,-1))


            # Compute loss
            print(f"states: {states.shape}")
            print(f"expert_actions: {expert_actions.shape}")
            loss = -policy.log_prob_t(states, expert_actions).mean()

            # Update the learner policy
            loss.backward()
            optimizer.step()

        # Evaluate the current learner policy
        learner_trajectory_dataset = MushroomRLTrajectoryDataset(mdp, learner_policy)
        data_loader = DataLoader(learner_trajectory_dataset, batch_size=batch_size, shuffle=True)

        # Collect new expert actions for the new states
        for state in learner_trajectory_dataset:
            state_np = state['state'].float().unsqueeze(0).cpu().numpy()
            expert_action = expert_policy.draw_action(state_np).astype(np.float32)
            state['action'] = torch.from_numpy(expert_action).squeeze().tolist()

        # Add the new data to the dataset
        dataset.trajectories.extend(learner_trajectory_dataset.trajectories)
        dataset.length += learner_trajectory_dataset.length

    return learner_policy


# 1. defining BCAgent and expert agent to be trained in dagger

In [35]:
env = "3dof-hit"

mdp = AirHockeyChallengeWrapper(env)
mdp.reset()

# policy can only output 1D actions (6,) ... they need to be recast in (2,3) shape later on
policy = GaussianTorchPolicy(Network, (12,), (6,), std_0=1., n_features=64, use_cuda=use_cuda)

policy = policy.load('dataset/hit_500_policy')
bc_agent = BCAgent(mdp.env_info, policy)

expert_agent = HittingAgent(mdp.env_info)
dataset = MushroomRLTrajectoryDataset(mdp, bc_agent, n_episodes=10)


100%|██████████| 10/10 [00:34<00:00,  3.44s/it]A


In [56]:
dagger_agent = train_dagger_agent(policy, expert_agent, mdp, dataset, num_iterations=2, batch_size=64, lr=0.001)

Exception in thread Thread-405:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-404:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 84, in _plan_trajectory_thread
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 84, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos,

states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])
states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])
states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])


Exception in thread Thread-604:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 84, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 135, in plan_ee_trajectory
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bezier_planner.t_final + 1e-6,
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 135, in <listcomp>
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bez

states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])


Exception in thread Thread-695:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 84, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 115, in plan_ee_trajectory
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bezier_planner.t_final + 1e-6,
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 115, in <listcomp>
    res = np.array([self.bezier_planner.get_point(t_i) for t_i in np.arange(0, self.bez

...

Exception in thread Thread-4070:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 84, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 121, in plan_ee_trajectory
    p = np.hstack([p, np.ones((p.shape[0], 1)) * self.ee_height])
  File "<__array_function__ internals>", line 200, in hstack
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/site-packages/numpy/core/shape_base.py", line 368, in hstack
    return _nx.concatenate(arrs, 0, dtype=dtype, casting=casting)
  File "

states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])


Exception in thread Thread-4141:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 84, in _plan_trajectory_thread
    ee_traj, hit_idx, q_anchor = self.plan_ee_trajectory(puck_pos, ee_pos)
  File "/Users/annetteader/PycharmProjects/air_hockey_challenge/examples/control/hitting_agent.py", line 121, in plan_ee_trajectory
    p = np.hstack([p, np.ones((p.shape[0], 1)) * self.ee_height])
  File "<__array_function__ internals>", line 200, in hstack
  File "/opt/anaconda3/envs/air_hockey_challenge/lib/python3.8/site-packages/numpy/core/shape_base.py", line 368, in hstack
    return _nx.concatenate(arrs, 0, dtype=dtype, casting=casting)
  File "

states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])
states: torch.Size([64, 12])
expert_actions: torch.Size([64, 6])


RuntimeError: shape '[64, -1]' is invalid for input of size 156

In [33]:
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)
optimizer = optim.Adam(policy.parameters(), lr=.001)

for i, data in enumerate(data_loader):
    states = data['state']
    for state in states:
        print(state.shape)
    # optimizer.zero_grad()
    # states = data['state'].float()
    # expert_actions = expert_agent.draw_action(states).float()

torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12